# 🚀 Google Colab Setup[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/14_best_practices/14_exercices_solutions.ipynb)**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')    !pip install -q numpy pandas matplotlib seaborn scikit-learn    !pip install -q mlflow fastapi pydantic uvicorn requests joblib    print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 14 - Solutions : Best Practices & MLOpsCe notebook contient des exercices pratiques pour consolider les concepts du Chapitre 14.**Instructions** :- Complétez les cellules marquées `# SOLUTION:# Remplacez cette ligne par votre code de solution`- Les solutions sont disponibles dans `14_exercices_solutions.ipynb`- N'hésitez pas à consulter la documentation (MLflow, FastAPI, Pydantic)**Note** : Certains exercices nécessitent l'exécution locale (serveur FastAPI, MLflow UI).---

## Setup Initial

In [ ]:
# Imports nécessairesimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsfrom sklearn.datasets import load_diabetes, load_winefrom sklearn.model_selection import train_test_split, GridSearchCVfrom sklearn.ensemble import RandomForestRegressor, RandomForestClassifierfrom sklearn.linear_model import LinearRegression, LogisticRegressionfrom sklearn.metrics import mean_squared_error, r2_score, accuracy_scoreimport mlflowimport mlflow.sklearnfrom mlflow.tracking import MlflowClientfrom fastapi import FastAPI, HTTPExceptionfrom pydantic import BaseModel, Field, validatorimport uvicornimport joblibimport requestsimport timefrom datetime import datetimeimport warningswarnings.filterwarnings('ignore')# Configurationsns.set_style('whitegrid')plt.rcParams['figure.figsize'] = (12, 6)np.random.seed(42)print("✓ Bibliothèques importées")

---## Exercice 1 : MLflow Tracking**Dataset** : Diabetes (régression)**Objectif** : Utiliser MLflow pour tracker plusieurs expériences.### 1.1 Configuration MLflow

In [ ]:
# Configurez MLflow# SOLUTION COMPLÈTE:# Implémentez le code selon les consignes# SOLUTION: Définissez le tracking URI (local)mlflow.set_tracking_uri("file:./mlruns")# SOLUTION: Créez ou définissez une expérienceexperiment_name = "diabetes-regression"mlflow.set_experiment(experiment_name)print(f"✓ Expérience MLflow : {experiment_name}")print(f"Tracking URI : {mlflow.get_tracking_uri()}")

### 1.2 Chargement des Données

In [ ]:
# Chargez le dataset Diabetes# SOLUTION COMPLÈTE:# Implémentez le code selon les consignesdiabetes = load_diabetes()X, y = diabetes.data, diabetes.target  # type: ignoreX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)print(f"Train : {X_train.shape}, Test : {X_test.shape}")

### 1.3 Entraînement avec Tracking MLflow

In [ ]:
# Entraînez plusieurs modèles avec tracking MLflow# SOLUTION COMPLÈTE:# Implémentez le code selon les consignesmodels = {    'LinearRegression': LinearRegression(),    'RandomForest_10': RandomForestRegressor(n_estimators=10, random_state=42),    'RandomForest_50': RandomForestRegressor(n_estimators=50, random_state=42),    'RandomForest_100': RandomForestRegressor(n_estimators=100, random_state=42)}for model_name, model in models.items():    # SOLUTION: Démarrez un run MLflow    with mlflow.start_run(run_name=model_name):        # SOLUTION: Loggez les paramètres        # mlflow.log_param("model_type", model_name)        # if hasattr(model, 'n_estimators'):        #     mlflow.log_param("n_estimators", model.n_estimators)                # SOLUTION: Entraînez le modèle        start_time = time.time()        # model.fit(X_train, y_train)        training_time = time.time() - start_time                # SOLUTION: Prédictions        # y_pred_train = model.predict(X_train)        # y_pred_test = model.predict(X_test)                # SOLUTION: Calculez les métriques        # train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))        # test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))        # train_r2 = r2_score(y_train, y_pred_train)        # test_r2 = r2_score(y_test, y_pred_test)                # SOLUTION: Loggez les métriques        # mlflow.log_metric("train_rmse", train_rmse)        # mlflow.log_metric("test_rmse", test_rmse)        # mlflow.log_metric("train_r2", train_r2)        # mlflow.log_metric("test_r2", test_r2)        # mlflow.log_metric("training_time", training_time)                # SOLUTION: Loggez le modèle        # mlflow.sklearn.log_model(model, "model")                print(f"{model_name} | Test RMSE: {test_rmse:.2f} | Test R²: {test_r2:.3f}")print("\n✓ Toutes les expériences sont enregistrées dans MLflow")print("Lancez 'mlflow ui' dans le terminal pour visualiser les résultats")

### 1.4 Recherche et Comparaison des Runs

In [ ]:
# Récupérez et comparez les runs# SOLUTION COMPLÈTE:# Implémentez le code selon les consignesclient = MlflowClient()# SOLUTION: Récupérez l'experiment# experiment = client.get_experiment_by_name(experiment_name)# runs = client.search_runs(experiment.experiment_id)# SOLUTION: Créez un DataFrame avec les résultatsresults = []# for run in runs:#     results.append({#         'run_name': run.data.tags.get('mlflow.runName', 'Unknown'),#         'test_rmse': run.data.metrics.get('test_rmse'),#         'test_r2': run.data.metrics.get('test_r2'),#         'training_time': run.data.metrics.get('training_time')#     })results_df = pd.DataFrame(results)results_df = results_df.sort_values('test_rmse')print("\nComparaison des modèles :")display(results_df)

### 1.5 Chargement d'un Modèle depuis MLflow

In [ ]:
# Chargez le meilleur modèle# SOLUTION COMPLÈTE:# Implémentez le code selon les consignes# SOLUTION: Récupérez le run_id du meilleur modèle# best_run_id = runs[0].info.run_id  # (après tri par test_rmse)# SOLUTION: Chargez le modèle# model_uri = f"runs:/{best_run_id}/model"# loaded_model = mlflow.sklearn.load_model(model_uri)# SOLUTION: Testez le modèle chargé# y_pred = loaded_model.predict(X_test)# rmse = np.sqrt(mean_squared_error(y_test, y_pred))# print(f"\nModèle chargé depuis MLflow | Test RMSE: {rmse:.2f}")

---## Exercice 2 : API REST avec FastAPI**Objectif** : Créer une API pour servir des prédictions.### 2.1 Définition des Schémas Pydantic

In [ ]:
# Définissez les schémas de validation# SOLUTION COMPLÈTE:# Implémentez le code selon les consignesclass DiabetesInput(BaseModel):    """    Schéma de validation pour les inputs du modèle Diabetes.    10 features : age, sex, bmi, bp, s1, s2, s3, s4, s5, s6    """    # SOLUTION: Définissez les 10 features avec contraintes    age: float = Field(..., ge=-3, le=3, description="Age (standardisé)")    sex: float = Field(..., ge=-3, le=3, description="Sex (standardisé)")    bmi: float = Field(..., ge=-3, le=3, description="Body Mass Index")    bp: float = Field(..., ge=-3, le=3, description="Blood Pressure")    s1: float = Field(..., ge=-3, le=3, description="Serum 1")    s2: float = Field(..., ge=-3, le=3, description="Serum 2")    s3: float = Field(..., ge=-3, le=3, description="Serum 3")    s4: float = Field(..., ge=-3, le=3, description="Serum 4")    s5: float = Field(..., ge=-3, le=3, description="Serum 5")    s6: float = Field(..., ge=-3, le=3, description="Serum 6")        class Config:        schema_extra = {            "example": {                "age": 0.05, "sex": 0.05, "bmi": 0.06, "bp": 0.02,                "s1": -0.04, "s2": -0.03, "s3": -0.00, "s4": -0.00,                "s5": 0.01, "s6": -0.04            }        }class PredictionOutput(BaseModel):    """Schéma de sortie."""    prediction: float    model_name: str    timestamp: strprint("✓ Schémas Pydantic définis")

### 2.2 Création de l'API FastAPI

In [ ]:
# Créez l'application FastAPI# SOLUTION COMPLÈTE:# Implémentez le code selon les consignes# Note : Ce code doit être exécuté dans un script Python séparé (api.py)# pour être lancé avec uvicorn# Exemple de structure :"""# api.pyfrom fastapi import FastAPI, HTTPExceptionfrom pydantic import BaseModel, Fieldimport joblibimport numpy as npfrom datetime import datetime# Créez l'appapp = FastAPI(title="Diabetes Prediction API", version="1.0")# SOLUTION: Chargez le modèle au démarragemodel = None  # joblib.load('diabetes_model.pkl')@app.get("/")def root():    return {"message": "Diabetes Prediction API", "version": "1.0"}@app.get("/health")def health_check():    # SOLUTION: Vérifiez que le modèle est chargé    return {"status": "healthy", "model_loaded": model is not None}@app.post("/predict", response_model=PredictionOutput)def predict(data: DiabetesInput):    # SOLUTION: Convertissez les données en array    # SOLUTION: Faites la prédiction    # SOLUTION: Retournez le résultat    pass@app.post("/predict/batch")def predict_batch(data: list[DiabetesInput]):    # SOLUTION: Prédictions en batch    pass"""print("Exemple de structure API créé")print("\nPour exécuter :")print("1. Sauvegardez le code ci-dessus dans 'api.py'")print("2. Sauvegardez un modèle : joblib.dump(model, 'diabetes_model.pkl')")print("3. Lancez : uvicorn api:app --reload")print("4. Documentation : http://localhost:8000/docs")

### 2.3 Sauvegarde du Modèle pour l'API

In [ ]:
# Sauvegardez un modèle entraîné# SOLUTION COMPLÈTE:# Implémentez le code selon les consignes# SOLUTION: Entraînez un modèlemodel_api = RandomForestRegressor(n_estimators=50, random_state=42)model_api.fit(X_train, y_train)# SOLUTION: Sauvegardez avec joblibjoblib.dump(model_api, 'diabetes_model.pkl')print("✓ Modèle sauvegardé dans 'diabetes_model.pkl'")

### 2.4 Test de l'API (après démarrage du serveur)

In [ ]:
# Testez l'API avec requests# SOLUTION COMPLÈTE:# Implémentez le code selon les consignes# Note : L'API doit être démarrée (uvicorn api:app)# SOLUTION: Test health check# response = requests.get("http://localhost:8000/health")# print(f"Health Check : {response.json()}")# SOLUTION: Test prédiction# test_data = {#     "age": 0.05, "sex": 0.05, "bmi": 0.06, "bp": 0.02,#     "s1": -0.04, "s2": -0.03, "s3": -0.00, "s4": -0.00,#     "s5": 0.01, "s6": -0.04# }# response = requests.post("http://localhost:8000/predict", json=test_data)# print(f"\nPrédiction : {response.json()}")print("\nPour tester l'API :")print("1. Assurez-vous que l'API est démarrée")print("2. Décommentez et exécutez le code ci-dessus")

---## Exercice 3 : Monitoring et Métriques**Objectif** : Ajouter du monitoring à l'API.### 3.1 Classe de Monitoring

In [ ]:
# Implémentez une classe de monitoring# SOLUTION COMPLÈTE:# Implémentez le code selon les consignesclass APIMonitor:    def __init__(self):        # SOLUTION: Initialisez les compteurs        self.total_requests = 0        self.total_errors = 0        self.response_times = []        self.predictions = []        self.start_time = time.time()        def log_request(self, response_time, success=True, prediction=None):        """Enregistre une requête."""        # SOLUTION: Incrémentez les compteurs        pass        def get_metrics(self):        """Retourne les métriques."""        # SOLUTION: Calculez les statistiques        uptime = time.time() - self.start_time                return {            "total_requests": self.total_requests,            "total_errors": self.total_errors,            "error_rate": self.total_errors / max(self.total_requests, 1),            "avg_response_time": np.mean(self.response_times) if self.response_times else 0,            "p95_response_time": np.percentile(self.response_times, 95) if self.response_times else 0,            "uptime_seconds": uptime,            "requests_per_second": self.total_requests / max(uptime, 1)        }# Testmonitor = APIMonitor()monitor.log_request(0.05, success=True, prediction=150.0)monitor.log_request(0.03, success=True, prediction=180.0)monitor.log_request(0.10, success=False)print("Métriques :")print(monitor.get_metrics())

### 3.2 Ajout du Monitoring à l'API

In [ ]:
# Code à ajouter dans api.py# SOLUTION COMPLÈTE:# Implémentez le code selon les consignes"""# Dans api.pymonitor = APIMonitor()@app.middleware("http")async def add_monitoring(request, call_next):    start_time = time.time()    response = await call_next(request)    response_time = time.time() - start_time        # SOLUTION: Loggez la requête    monitor.log_request(response_time, success=response.status_code < 400)        return response@app.get("/metrics")def get_metrics():    return monitor.get_metrics()"""print("Code de monitoring pour l'API")

---## Exercice 4 : Load Testing**Objectif** : Tester les performances de l'API sous charge.### 4.1 Script de Load Testing

In [ ]:
# Implémentez un load test simple# SOLUTION COMPLÈTE:# Implémentez le code selon les consignesdef load_test(url, n_requests=100, n_workers=10):    """    Effectue un load test sur l'API.    Args:        url: URL de l'endpoint        n_requests: Nombre total de requêtes        n_workers: Nombre de threads concurrents    """    from concurrent.futures import ThreadPoolExecutor    import time        test_data = {        "age": 0.05, "sex": 0.05, "bmi": 0.06, "bp": 0.02,        "s1": -0.04, "s2": -0.03, "s3": -0.00, "s4": -0.00,        "s5": 0.01, "s6": -0.04    }        response_times = []    errors = 0        def send_request():        # SOLUTION: Envoyez une requête et mesurez le temps        start = time.time()        try:            response = requests.post(url, json=test_data, timeout=5)            response_time = time.time() - start            return response_time, response.status_code == 200        except Exception as e:            return time.time() - start, False        # SOLUTION: Exécutez les requêtes en parallèle    start_time = time.time()        with ThreadPoolExecutor(max_workers=n_workers) as executor:        results = list(executor.map(lambda _: send_request(), range(n_requests)))        total_time = time.time() - start_time        # SOLUTION: Analysez les résultats    response_times = [r[0] for r in results]    errors = sum(1 for r in results if not r[1])        print(f"\n=== Load Test Results ===")    print(f"Total requests: {n_requests}")    print(f"Concurrent workers: {n_workers}")    print(f"Total time: {total_time:.2f}s")    print(f"Requests/sec: {n_requests / total_time:.2f}")    print(f"\nResponse times:")    print(f"  Min: {min(response_times)*1000:.2f}ms")    print(f"  Max: {max(response_times)*1000:.2f}ms")    print(f"  Mean: {np.mean(response_times)*1000:.2f}ms")    print(f"  P95: {np.percentile(response_times, 95)*1000:.2f}ms")    print(f"\nErrors: {errors} ({errors/n_requests*100:.1f}%)")        return response_times, errors# SOLUTION: Exécutez le load test (après avoir démarré l'API)# response_times, errors = load_test("http://localhost:8000/predict", n_requests=100, n_workers=10)print("\nPour exécuter le load test :")print("1. Démarrez l'API")print("2. Décommentez et exécutez la ligne ci-dessus")

---## Exercice 5 : Model Registry avec MLflow**Objectif** : Gérer le cycle de vie des modèles (dev → staging → production).### 5.1 Enregistrement dans le Model Registry

In [ ]:
# Enregistrez un modèle dans le Model Registry# SOLUTION COMPLÈTE:# Implémentez le code selon les consignes# SOLUTION: Entraînez un modèlemodel_registry = RandomForestRegressor(n_estimators=100, random_state=42)model_registry.fit(X_train, y_train)# SOLUTION: Enregistrez le modèle avec MLflowwith mlflow.start_run(run_name="production_candidate"):    # Loggez les métriques    y_pred = model_registry.predict(X_test)    rmse = np.sqrt(mean_squared_error(y_test, y_pred))    r2 = r2_score(y_test, y_pred)        mlflow.log_metric("test_rmse", rmse)    mlflow.log_metric("test_r2", r2)        # SOLUTION: Enregistrez dans le Model Registry    # mlflow.sklearn.log_model(    #     model_registry,    #     "model",    #     registered_model_name="diabetes_predictor"    # )        print(f"✓ Modèle enregistré | RMSE: {rmse:.2f} | R²: {r2:.3f}")print("\nVérifiez dans MLflow UI : http://localhost:5000")

### 5.2 Transition de Stage

In [ ]:
# Promouvez un modèle vers staging puis production# SOLUTION COMPLÈTE:# Implémentez le code selon les consignesclient = MlflowClient()# SOLUTION: Récupérez la dernière version du modèle# model_name = "diabetes_predictor"# latest_version = client.get_latest_versions(model_name)[0]# SOLUTION: Transition vers "Staging"# client.transition_model_version_stage(#     name=model_name,#     version=latest_version.version,#     stage="Staging"# )# print(f"✓ Modèle v{latest_version.version} → Staging")# SOLUTION: Après validation, transition vers "Production"# client.transition_model_version_stage(#     name=model_name,#     version=latest_version.version,#     stage="Production"# )# print(f"✓ Modèle v{latest_version.version} → Production")print("\nWorkflow Model Registry :")print("1. Entraînez et enregistrez le modèle")print("2. Testez en staging")print("3. Promouvez en production si validé")

---## Exercice 6 : Questions de Réflexion

**Question 1** : Pourquoi est-il important de valider les inputs avec Pydantic dans une API ?**RÉPONSE:**\n\n*Complétez avec votre réponse basée sur le contexte de l'exercice*---**Question 2** : Quelles métriques devriez-vous monitorer pour une API de ML en production ?**RÉPONSE:**\n\n*Complétez avec votre réponse basée sur le contexte de l'exercice*---**Question 3** : Quelle est la différence entre staging et production dans le Model Registry ?**RÉPONSE:**\n\n*Complétez avec votre réponse basée sur le contexte de l'exercice*---**Question 4** : Comment géreriez-vous un rollback si le nouveau modèle en production performe mal ?**RÉPONSE:**\n\n*Complétez avec votre réponse basée sur le contexte de l'exercice*

---## ConclusionFélicitations pour avoir complété ces exercices !**Points clés à retenir** :- MLflow permet de tracker et comparer facilement les expériences- FastAPI + Pydantic offrent une validation robuste et une documentation automatique- Le monitoring est essentiel pour détecter les problèmes en production- Le Model Registry facilite la gestion du cycle de vie des modèles- Le load testing permet d'identifier les goulots d'étranglement**Prochaines étapes** :- Consultez les solutions dans `14_exercices_solutions.ipynb`- Déployez votre API sur un serveur cloud (AWS, GCP, Azure)- Explorez Docker et Kubernetes pour le déploiement- Mettez en place du CI/CD (GitHub Actions, GitLab CI)- Félicitations, vous avez terminé le cours de Machine Learning !---